<a href="https://colab.research.google.com/github/ammarisme/covid-19/blob/master/CV19_result_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline
!pip install torch-geometric \
  torch-sparse==latest+cu101 \
  torch-scatter==latest+cu101 \
  torch-cluster==latest+cu101 \
  -f https://pytorch-geometric.com/whl/torch-1.5.0.html

Looking in links: https://pytorch-geometric.com/whl/torch-1.5.0.html
     |████████████████████████████████| 133kB 2.7MB/s 
     |████████████████████████████████| 21.9MB 1.4MB/s 
     |████████████████████████████████| 12.3MB 53.6MB/s 
     |████████████████████████████████| 18.2MB 1.3MB/s 
     |████████████████████████████████| 235kB 8.7MB/s 
     |████████████████████████████████| 51kB 6.1MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-1.4.3-cp36-none-any.whl size=234873 sha256=e8620c481d785919f634c36ad9ccfa2d744995a0e4d9dda66c01206dd7d07639
  Stored in directory: /root/.cache/pip/wheels/e2/c1/09/8693feee3f97e440d68b09abfca8b4c1e97150ace350b5003f
Successfully built torch-geometric


In [2]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
from functools import reduce
import os

import torch
from torch_geometric.data import Data, DataLoader, InMemoryDataset

import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd

from os import listdir
from os.path import isfile, join

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('running on '+ ("GPU" if torch.cuda.is_available() else "CPU"))

running on CPU


In [4]:
from google.colab import drive
drive.mount('/content/drive')
PATH = '/content/drive/My Drive/covid'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
def loss_file_search(mypath):
  losses_files = [f for f in listdir(mypath) if ("losses" in f)]
  ax_paths = []
  for loss_file in losses_files:
    losses = np.load(join(mypath, loss_file), allow_pickle=True)
    training_loss = np.array(losses.tolist()['losses']).T[0]
    ax = plt.plot(training_loss)
    filepath = mypath
    ax_paths.append((ax, filepath))
  
  directories = [f for f in listdir(mypath) if os.path.isdir(join(mypath, f))]
  for directory in directories:
    ax_paths.append(loss_file_search(mypath+'/'+directory))
  
  return ax_paths
  

In [7]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import ylim

import matplotlib.patches as mpatches


root = PATH+'/models__rnn/4'
#ylim(top=5.08, bottom=5.07)#, ylim_bottom=5.0
#ax_paths = loss_file_search(root)
"""axes = [ax_path[0][0] for ax_path in ax_paths]
paths = [ax_path[0][1][-1] for ax_path in ax_paths]
colors = {
    0 :"red",
    3 : "green",
    4 : "blue",
    6: "orange"
}
plt.legend(handles=[
                    mpatches.Patch(color=colors[int(path)], label=str(path)) for path in paths])
plt.show()
"""

'axes = [ax_path[0][0] for ax_path in ax_paths]\npaths = [ax_path[0][1][-1] for ax_path in ax_paths]\ncolors = {\n    0 :"red",\n    3 : "green",\n    4 : "blue",\n    6: "orange"\n}\nplt.legend(handles=[\n                    mpatches.Patch(color=colors[int(path)], label=str(path)) for path in paths])\nplt.show()\n'

In [0]:
class CovidDataSet(InMemoryDataset):
    def __init__(self, root, input_sequence, output_sequence, transform=None, pre_transform=None):
        super(CovidDataSet, self).__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_dir(self):
      if os.path.exists(self.root+PROCESSED_DIR):
        return self.root+'/cleaned'
      else:
        os.mkdir(self.root+PROCESSED_DIR)
        return self.root+'/cleaned'
        
    @property
    def processed_dir(self):
      if os.path.exists(self.root+PROCESSED_DIR):
        return self.root+PROCESSED_DIR
      else:
        os.mkdir(self.root+PROCESSED_DIR)
        return self.root+PROCESSED_DIR

    @property
    def raw_file_names(self):
      mypath = self.raw_dir
      filenames = [f for f in listdir(mypath) if isfile(join(mypath, f))]
      return filenames

    @property
    def processed_file_names(self):
        return ['processed.dt']

    def download(self):
        pass
    
    def process(self):
        pass

In [21]:
configuration = {
    'input_sequence_len' : 10,
    'output_sequence_len' : 10,
    'training_batch_size' : 1024,
    'training_dataset_length' :32768,
    'validation_batch_size' : 1024,
    'yhat_size' : 2,
    'feature_len' : 8,
    'output_size' : 2,
}


INPUT_ROOT = PATH+'/input_mix'
DATA_TAG = "seq2seq_"+str(configuration['input_sequence_len'])+'_'+str(configuration['output_sequence_len'])
PROCESSED_DIR = '/processed_'+DATA_TAG

validation_dataset = CovidDataSet(INPUT_ROOT+'/validation', configuration['input_sequence_len'], configuration['output_sequence_len'])
validation_dataset = validation_dataset.shuffle()
validation_dataset = validation_dataset[3000:]
validation_dataloader = DataLoader(validation_dataset,batch_size=configuration['validation_batch_size'])

print('batches validation :', len(validation_dataloader))
print('dataset length validation :', len(validation_dataset))

batches validation : 7
dataset length validation : 6260


In [0]:
class RNNModel(nn.Module):
    def __init__(self, input_size, parameter_sizes, repeats ,output_size):
        super(RNNModel, self).__init__()
        self.input_size = input_size
        self.repeater_input_size = parameter_sizes[0]
        self.hidden_size = parameter_sizes[1]
        self.repeats = repeats
        self.output_size = output_size

        self.fc1 = nn.Linear(input_size, self.repeater_input_size)
        self.relu_activation = nn.ReLU()

        self.layers = dict()

        
        k = 0
        for i in range(repeats):
          i = i+k
          self.layers['fc_'+str(i)] = nn.Linear(self.repeater_input_size, self.hidden_size)
          self.layers['gru_'+str(i+1)] = nn.GRU(self.hidden_size, self.hidden_size)
          self.layers['fc_'+str(i+2)] = nn.Linear(self.hidden_size, self.repeater_input_size)
          k+=2

        self.module_list = nn.ModuleDict(self.layers)

        self.fc2 = nn.Linear(self.repeater_input_size, output_size)
        
    def forward(self, input, hidden):
      output = self.fc1(input)
      output = self.relu_activation(output)
      print(output.sum())
      k = 0
      for i in range(self.repeats):
        i = i+k
        output = self.layers['fc_'+str(i)](output)
        output = self.relu_activation(output)

        output, hidden[i-k] = self.layers['gru_'+str(i+1)](output, hidden[i-k])#should be different. check the nlp page
        output = self.relu_activation(output)
        hidden[i-k] = self.relu_activation(hidden[i-k])

        output = self.layers['fc_'+str(i+2)](output)
        output = self.relu_activation(output)
        k +=2

      output = self.fc2(output)
      output = self.relu_activation(output)

      return output, hidden

    def initHidden(self, batch_size):
        return torch.zeros(1, batch_size, self.hidden_size, device=device)

In [72]:
def predict(input_tensor, rnn_model):
  with torch.no_grad():
    input_tensor = input_tensor.type(torch.FloatTensor)

    rnn_model_hidden = []
    for i in range(5):
      rnn_model_hidden.append(rnn_model.initHidden(1))

    validation_loss = 0
    
    outputs = []

    for ei in range(configuration['input_sequence_len']):
      input_tensor_seq = input_tensor.view(configuration['input_sequence_len'],1, configuration['feature_len'])[ei]
      
      input_tensor_seq = input_tensor_seq.view(1, 1, configuration['feature_len']).to(device)
      rnn_model_output, rnn_model_hidden = rnn_model(
          input_tensor_seq, rnn_model_hidden)
      outputs.append(rnn_model_output.detach().numpy())

  return outputs
model = RNNModel(**{
         'input_size': 8,
         'parameter_sizes': [256, 128] ,
         'repeats' : 5 ,
         'output_size': 2
         })

model.eval()
out1 = np.array(predict(torch.tensor(validation_dataset[50].x), model)).squeeze()
#out2 = np.array(predict(torch.tensor(validation_dataset[100].x), model)).squeeze()
#if np.sum(out1 - out2)==0: print('Model is broken')

tensor(2.6286e+08)
tensor(2.6286e+08)
tensor(2.6286e+08)
tensor(2.6286e+08)
tensor(2.6286e+08)
tensor(2.6286e+08)
tensor(2.6286e+08)
tensor(2.6286e+08)
tensor(2.6286e+08)
tensor(2.6286e+08)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


In [104]:
print('weights',np.mean(model.fc1.weight.detach().numpy()))
print('bias',np.mean(model.fc1.bias.detach().numpy()))

weights 0.0003171371
bias -0.0071668727


In [0]:
model = RNNModel(**{
         'input_size': 8,
         'parameter_sizes': [256, 128] ,
         'repeats' : 5 ,
         'output_size': 2
         })
#model_statedict = torch.load(root+'/model_manual_save_0628_5.pt', map_location=torch.device('cpu'))
#model.load_state_dict(model_statedict)
#model.train()

In [57]:
out1 = np.array(predict(torch.tensor(validation_dataset[50].x), model)).squeeze()
out2 = np.array(predict(torch.tensor(validation_dataset[100].x), model)).squeeze()
if np.sum(out1 - out2)==0: print('Model is broken')

Model is broken


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [0]:
def get_all_directories(mypath):  
  directories= [f for f in listdir('/'+mypath) if os.path.isdir(join(mypath, f))]
  for directory in directories:
    get_all_directories(directory)
    #loss_file_search(mypath+'/'+directory)

get_all_directories(PATH)

FileNotFoundError: ignored

In [0]:
def loss_analysis(losses):
 print('loss analysis') 

['CV19 Cleaning.ipynb',
 'CV19_Synthesis.ipynb',
 '_Common.ipynb',
 'common (1).py',
 'common.py',
 'CV19Net.ipynb',
 'rnn_model_configurations_1001_443838.csv',
 'rnn_model_configurations_1001_165567.csv',
 'rnn_model_configurations_1001_353166.csv',
 'rnn_model_configurations_1001_57460.csv',
 'rnn_model_configurations_1001_981028.csv',
 'CV19 Dataset Explore.ipynb',
 'CV19_Dataset.ipynb',
 'RNN-CV19Net.ipynb',
 'CV19-result-analysis.ipynb']